In [ ]:
!pip install -U scikit-learn

In [ ]:
!pip install pandas
!pip install --timeout=600 tensorflow[and-cuda]
!pip install numpy 


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from keras.applications.efficientnet import EfficientNetB0
from PIL import Image
import numpy as np 
import keras.backend as K
from keras import metrics,layers,models
import tqdm


<h3> GPU Testing Tensorflow Compatibility </h3>

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Print the GPU devices
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("GPU Device Name:", physical_devices[0].name)
else:
    print("No GPU devices found.")


    # Check devices
devices = tf.config.list_physical_devices()
print(devices)


<h3> IDK WHAT THIS IS </h3>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Input, Concatenate, Lambda
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
import numpy as np 
from PIL import Image
import os
import random
import pandas as pd

# Assuming you have a function to convert an image to a tensor
def convert_image_to_tensor(path):
    img = Image.open(path)
    img = img.resize((224, 224))
    img = np.asarray(img)
    return img

def get_files_list(path_of_file):
    file_list = os.listdir(path_of_file)
    file_list = [os.path.join(path_of_file, i) for i in file_list]
    shuffled_file_list = [item for item in file_list for _ in range(1)]
    random.shuffle(shuffled_file_list)
    for f in shuffled_file_list:
        file_list.append(f)
        
    return file_list

left_images_path = 'left/left/'  # path for all left side images
right_images_path = 'right/right/'  # path for all right side images
data_set_raw = pd.DataFrame(columns=['left_image', 'right_image', 'label'])

left_images = get_files_list(left_images_path)
right_images = get_files_list(right_images_path)

data_set_raw['left_image'] = left_images
data_set_raw['right_image'] = right_images

data_set_raw['label'] = 0
data_set_raw.loc[data_set_raw['left_image'].apply(lambda x: x.split('/')[-1]) == data_set_raw['right_image'].apply(lambda x: x.split('/')[-1]), 'label'] = 1

X_left_paths = data_set_raw['left_image']
X_right_paths = data_set_raw['right_image']
Y = np.array(data_set_raw['label'].to_list()).reshape(-1, 1)

# Split the data into training and validation sets
X_left_train, X_left_val, X_right_train, X_right_val, Y_train, Y_val = train_test_split(
    X_left_paths, X_right_paths, Y, test_size=0.2, random_state=42
)

# Input layers
input_left = Input(shape=(224, 224, 3))
input_right = Input(shape=(224, 224, 3))

# Build Siamese model
def build_siamese_model():
    input_val = Input(shape=(224, 224, 3))
    base_model = EfficientNetB0(include_top=False, input_tensor=input_val, weights='imagenet')

    for layer in base_model.layers[:int(len(base_model.layers) * 0.75)]:
        layer.trainable = False

    encoded_left = base_model(input_left)
    encoded_right = base_model(input_right)

    flat_encoded_left = Flatten()(encoded_left)
    flat_encoded_right = Flatten()(encoded_right)

    distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([flat_encoded_left, flat_encoded_right])
    dense_output = Dense(1, activation='sigmoid')(distance)

    model = tf.keras.models.Model(inputs=[input_left, input_right], outputs=dense_output)
    return model

# Create the Siamese model
siamese_model = build_siamese_model()

# Compile the model with contrastive loss
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
siamese_model.summary()

# Train the Siamese model over batches
batch_size = 64  # Set your desired batch size
total_epochs = 20  # Set the total number of epochs

try:
    siamese_model.load_weights('siamese_final.h5')  # Adjust the filename based on your requirements
    print('Loaded Model')
    starting_epoch=5
except:
    print('Resetting from Scratch')
    starting_epoch=0
    


for epoch in range(total_epochs):
    print(f'Epoch Number: {epoch + 1}')
    
    # Create a generator for labels
    label_generator = iter(Y_train)
    
    # Calculate the number of batches
    num_batches = len(X_left_train) // batch_size
    
    # Iterate over batches
    for i in range(num_batches):
        # Extract a batch of left and right images
        left_batch_paths = X_left_train[i * batch_size: (i + 1) * batch_size]
        right_batch_paths = X_right_train[i * batch_size: (i + 1) * batch_size]

        # Load and preprocess images
        left_batch = np.array([convert_image_to_tensor(img_path) for img_path in left_batch_paths])
        right_batch = np.array([convert_image_to_tensor(img_path) for img_path in right_batch_paths])

        # Extract labels for the current batch
        label_batch = np.array([next(label_generator) for _ in range(batch_size)])

        # Train the Siamese model
        history = siamese_model.train_on_batch([left_batch, right_batch], label_batch)

        # Print batch-wise metrics
        print(f'Batch {i + 1}/{num_batches} - Loss: {history[0]}, Accuracy: {history[1]}')


In [ ]:
def convert_image_to_tensor(path,show_flag=0):
    img = Image.open(path)
    img = img.resize((224, 224))
    if show_flag==1:
        display(img)
    img = np.asarray(img)
    return img


siamese_model.predict([convert_image_to_tensor('left/exp_4.jpg',1).reshape(1,224,224,3),convert_image_to_tensor('right/exp_3.jpg',1).reshape(1,224,224,3)])

In [ ]:
siamese_model.save_weights('siamese_final.h5')